In [3]:
!pip install openai

In [29]:
import os
from openai import OpenAI
from pathlib import Path

api_key = "" 
client = OpenAI(api_key=api_key)


In [31]:
client.files.create(
    file=Path("quiz_data.jsonl"),
    purpose="fine-tune",
)

FileObject(id='file-au3KivxsUPcQr16fXx6br48Y', bytes=14252, created_at=1731485466, filename='quiz_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [35]:
response = client.fine_tuning.jobs.create(
    training_file = "file-au3KivxsUPcQr16fXx6br48Y",
    model = "gpt-4o-2024-08-06",
    hyperparameters={
    "n_epochs":2
  }
)
print(response)

FineTuningJob(id='ftjob-lit0RjWu67OhgJpZuDhxR1ce', created_at=1731485567, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-mzKEKjPTDzVL4GIR0reinzMC', result_files=[], seed=1801962442, status='validating_files', trained_tokens=None, training_file='file-au3KivxsUPcQr16fXx6br48Y', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)


In [45]:
response = client.fine_tuning.jobs.retrieve("ftjob-lit0RjWu67OhgJpZuDhxR1ce")

if response.status == "succeeded" :
    print(response.fine_tuned_model)
else:
    print(response)

ft:gpt-4o-2024-08-06:hm::AT2tXRi6


In [198]:
def generate_quiz(topic, max_tokens=120):
    prompt = f"Create a simple 3-5 question MCQ quiz on the following question: {topic}, do not add anything other than the questions\nQuiz:"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AT2tXRi6",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=1,
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content
    

def quiz_answers(prompt , max_tokens =100):
    prompt = f"Give me the answers to the quiz, {quiz}, give me only the correct answers in a single line, nothing else"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AT2tXRi6",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=1,
        temperature=0
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def grade_quiz(quiz_answer, user_response, max_tokens =120):
    prompt = f"Mark the following quiz answers: \nCorrect answers: {quiz_answer} \nUser's answers: {user_response} \nScore the user's answers out of 100%, only return the score as a percentage."
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AT2tXRi6",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=1,
        temperature=0
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def assign_skill_level(quiz_score):
  if quiz_score >= 90:
    return "Advanced"
  elif quiz_score >= 70:
    return "Intermediate"
  else:
    return "Beginner"

def generate_additional_quiz(topic, skill_level, max_tokens=130):
    prompt = f"Create a 2 {skill_level} level MCQ questions on {topic}, do not add anything other than the question"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AT2tXRi6",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=1
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def explain_answers(quiz, quiz_answers, user_response, max_tokens = 150):
    prompt = f"Explain any question that the user got wrong in this quiz \n {quiz}, \n these are the answers \n {quiz_answers} and this is the user's answers \n {user_response} \n explain how to get the answers \n ONLY explain incorrect answers"
    response = client.chat.completions.create(
    model="ft:gpt-4o-2024-08-06:hm::AT2tXRi6",
    messages=[{"role":"user","content":prompt}],
    n=1,
    temperature = 0.5
    )
    return response.choices[0].message.content

def respond_to_query(query, max_tokens=100):
    prompt = f"{query}"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AT2tXRi6",
        messages=[{"role": "user", "content": prompt}],
        max_tokens = max_tokens,
        n=1
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

In [196]:
topic = "Basic Addition"
quiz = generate_quiz(topic)
print(quiz)

1. What is 2 + 2?
A) 3
B) 4
C) 5
D) 6

2. What is 3 + 5?
A) 7
B) 8
C) 9
D) 10

3. What is 7 + 1?
A) 7
B) 8
C) 9
D) 10

4. What is 4 + 6?
A) 9
B) 10
C) 11
D) 12




In [123]:
quiz_answer = quiz_answers(quiz)
print(quiz_answer)

B, B, B, B


In [165]:
user_response = ['a','b','b','b']
response = grade_quiz(quiz_answer,user_response)
print(response)
quiz_score = float(response.strip('%'))
skill_level = assign_skill_level(quiz_score)
print(skill_level)

75%
Intermediate


In [204]:
response = explain_answers(quiz,quiz_answers,user_response)
print(response)

To find the correct answer for question 1, add 2 + 2 to get 4, which is option B.


In [169]:
additiona_quiz = generate_additional_quiz(topic, skill_level)
print(additiona_quiz)


1. What is the sum of 256 + 178?
A) 424
B) 434
C) 444
D) 454

2. What is 789 + 123?
A) 902
B) 912
C) 905
D) 910
